Задание взять данные из https://www.kaggle.com/datasets/mrapplexz/bashim-quotes обучить модель GPT для генерации своих цитат

взять новостные данные из https://github.com/natasha/corus load_lenta2 нам понадобится сам текст и заголовок обучить модель T5/ или GPT для генерации заголовков для статей

In [9]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [10]:
from google.colab import files
upload = files.upload

In [11]:
!ls -la

total 39612
drwxr-xr-x 1 root root     4096 Jan 21 11:23 .
drwxr-xr-x 1 root root     4096 Jan 21 11:13 ..
drwxr-xr-x 4 root root     4096 Jan 19 14:35 .config
drwxr-xr-x 3 root root     4096 Jan 21 11:22 data
-rw-r--r-- 1 root root 40531099 Jan 21 11:26 dataset.jsonl
drwxr-xr-x 2 root root     4096 Jan 21 11:21 .ipynb_checkpoints
drwxr-xr-x 1 root root     4096 Jan 19 14:35 sample_data


In [12]:
!mkdir data

mkdir: cannot create directory ‘data’: File exists


In [13]:
!mv 'dataset.jsonl' 'data/dataset.jsonl'

In [14]:
DATASET_PATH = 'data/dataset.jsonl'
with open(DATASET_PATH) as f:
  df = pd.read_json(DATASET_PATH, lines=True).set_index('id')

In [15]:
df.head()

,date,rating,text
id,,,
1,2004-08-30 11:24:00+00:00,22010.0,"<Ares> ppdv, все юниксы очень дружелюбны.. они..."
2,2004-08-30 11:25:00+00:00,25105.0,<томатик_рад> а ты не чувствуешь красоту мира?...
3,2004-08-30 11:27:00+00:00,7192.0,"<Дор> ""мышка, почему у тебя такие большие глаз..."
4,2004-08-30 11:28:00+00:00,29169.0,"<PPDV[os2]> ""Мальчики, вы что больные, бегать ..."
5,2004-08-30 11:26:00+00:00,7140.0,<Ohtori_Akio> мы - как разработчики - живём с ...


In [16]:
pd.set_option('display.max_colwidth', 150)

In [17]:
df['text'][2]

'<томатик_рад> а ты не чувствуешь красоту мира?\n<fox> честно говоря, я сейчас чувствую только отсутствие http.\n<томатик_рад> не туда смотришь, глянь вокруг!\n<fox> как я гляну, если http не работает? :/'

In [18]:
!wget https://gist.githubusercontent.com/avidale/d3da0ded85a4a16db6eb84d8331638ce/raw/a188084e5ef37b43b01fef0534b55c865b9a569e/tweets.txt

--2023-01-21 11:29:18--  https://gist.githubusercontent.com/avidale/d3da0ded85a4a16db6eb84d8331638ce/raw/a188084e5ef37b43b01fef0534b55c865b9a569e/tweets.txt
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4659 (4.5K) [text/plain]
Saving to: ‘tweets.txt’

tweets.txt          100%[===================>]   4.55K  --.-KB/s    in 0s      

2023-01-21 11:29:18 (74.7 MB/s) - ‘tweets.txt’ saved [4659/4659]



In [19]:
!ls

data  sample_data  tweets.txt


In [20]:
with open('tweets.txt', 'r') as f:
  tweets = f.read().strip().split('\n\n')
  print(len(tweets))
  for i in range(3):
    print(tweets[i])

26
Соловьев наконец-то вышел на новый уровень - теперь его стали банить и в офлайне
Дарим мы тебе бутылку игристого вина. Пить тебе еще рано, но встретиться с ней за некоторые преступления ты уже можешь. ПОЗ-ДРАВ-ЛЯ-ЕМ!
Да. Еще очень многие помнят, что такое госплан, как планировалось, талоны на еду, очереди, дефицит, выездные визы. Но спасибо, что напомнил


In [21]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 65.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 105.8 MB/s eta 0:00:00


In [22]:
import torch

In [23]:
device = torch.device("cuda")

In [24]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [25]:
from transformers.pipelines.base import AutoModel
model_name = 'sberbank-ai/rugpt3large_based_on_gpt2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

Downloading:   0%|          | 0.00/609 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.71M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/3.14G [00:00<?, ?B/s]

In [26]:
import random

In [27]:
sep = '\n***\n'

In [28]:
prefix = sep.join([''] + random.sample(tweets, k=5) + [''])
tokens = tokenizer(prefix, return_tensors='pt')
tokens = {k: v.to(model.device) for k, v in tokens.items()}
end_token_id = tokenizer.encode('***')[0]
print(prefix)


***
Квартир ветеранам и достойную пенсию никто не гарантирует, но зато парад будет однозначно
***
Теперь главное, чтобы все дожили до второго чтения
***
Да. Еще очень многие помнят, что такое госплан, как планировалось, талоны на еду, очереди, дефицит, выездные визы. Но спасибо, что напомнил
***
Есть подозрение, что те самые 400 кг порошка хранятся в кабинете Володина под его чутким контролем. Иначе эти фантазии не объяснить
***
И немножечко зиганул
***



In [34]:
size = tokens['input_ids'].shape[1]
output = model.generate(
    **tokens,
    do_sample=False,
    max_length=size+128,
    repetition_penalty=4.2,
    temperature=0.7,
    num_beams=10,
)
decoded = tokenizer.decode(output[0])
result = decoded[len(prefix):]
print(result)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


А вот это уже интересно<s>
Оригинал взят у в postКак-то так получилось, что за последние несколько месяцев я ни разу не писала о том, что происходит в моей жизни. Наверное, из-за того, что мне просто некогда было этим заниматься. А может быть, потому, что слишком много всего навалилось: работа, учеба, домашние хлопоты... В общем, если вы хотите знать мое мнение по поводу происходящих событий, то вам сюда - http://www.liveinternet.ru/users/4052155/post428901643/?utm_


In [35]:
size = tokens['input_ids'].shape[1]
output = model.generate(
    **tokens,
    do_sample=False,
    max_length=size+128,
    repetition_penalty=4.2,
    temperature=1.3,
    num_beams=7,
    no_repeat_ngram_size=5
)
decoded = tokenizer.decode(output[0])
result = decoded[len(prefix):]
print(result)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


А вот это уже интересно...<s>
Оригинал взят у в postУважаемые читатели моего блога!
В связи с тем, что мой блог заблокирован администрацией ЖЖ по требованию Роскомнадзора, я вынужден обратиться к вам с просьбой о перепосте этого поста.
Я считаю, что этот пост должен быть размещен здесь: http://www.livejournal.com/support/faqbrowse.bml?faqid=10345678#cutid1
Пожалуйста, помогите мне распространить информацию об этом посте среди тех, кому небезразлич


In [36]:
!pip install razdel networkx pymorphy2[fast] nltk rouge==0.3.1
!pip install --upgrade datasets tqdm transformer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 KB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 55.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 371.1/371.1 KB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for DAWG: filename=DAWG-0.8.0-cp38-cp38-linux_x86_64.whl size=1312371 sha256=99ed0851f624e29aa465809d207cc1334ff0ca8c7ade66c8412080b2877a2709
  Stored in directory: /root/.cache/pip/wheels/1c/e6/8f/313a7ccc57b29a7affb7205664277a1d5ebe73bf600a69a615
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=cc7639a5ac07877015aebec424f24fe8e0ab1c7e16d26088481398106872e319
  Stored in directory: /root/.cache/pip/wheels/56/ea/58/ead137b087d9e326852a851351d1debf4ada529b6ac0ec4e8c
Successfully built DAWG docopt
Looking in indexes: ht

In [37]:
!ls

data  sample_data  tweets.txt


In [39]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached datasets-2.8.0-py3-none-any.whl (452 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 17.5 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [40]:
from datasets import load_dataset

dataset_train = load_dataset('IlyaGusev/gazeta', revision="v1.0", split= 'train[:10%]')
dataset_test = load_dataset('IlyaGusev/gazeta', revision="v1.0", split= 'test[:10%]')

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/52400 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5770 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5265 [00:00<?, ? examples/s]

Dataset gazeta downloaded and prepared to /root/.cache/huggingface/datasets/IlyaGusev___gazeta/default/1.0.0/ef9349c3c0f3112ca4036520d76c4bc1b8a79d30bc29643c6cae5a094d44e457. Subsequent calls will reuse this data.


In [41]:
dataset_train

Dataset({
    features: ['text', 'summary', 'title', 'date', 'url'],
    num_rows: 5240
})

In [42]:
dataset_test


Dataset({
    features: ['text', 'summary', 'title', 'date', 'url'],
    num_rows: 577
})

In [43]:
dataset_test['summary'][0]

'В NASA назвали четыре миссии в дальний космос, которые в этом десятилетии могут быть запущены американцами. Среди них — две миссии по изучению Венеры, полет к спутнику Юпитера и экспедиция к Тритону, спутнику Нептуна.'

In [44]:
dataset_test['title'][0]

'Венера, Ио или Тритон: куда полетит NASA'

In [45]:
model_name = "IlyaGusev/rut5_base_sum_gazeta"

In [46]:
def len_tok(text):
    return len(text.split())

In [47]:
max_len_sum, max_len_tl = max(map(len_tok, dataset_train['summary'])), max(map(len_tok, dataset_train['title']))
max_len_sum, max_len_tl

(75, 18)

In [48]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    tokenized_input = tokenizer(batch['summary'], padding='max_length', truncation=True, max_length=max_len_sum)
    tokenized_label = tokenizer(batch['title'], padding='max_length', truncation=True, max_length=max_len_tl)

    tokenized_input['labels'] = tokenized_label['input_ids']

    return tokenized_input

dataset_train = dataset_train.map(tokenize, batched=True, batch_size=8)
dataset_test = dataset_test.map(tokenize, batched=True, batch_size=8)

dataset_train.set_format('numpy', columns=['input_ids', 'attention_mask', 'labels'])
dataset_test.set_format('numpy', columns=['input_ids', 'attention_mask', 'labels'])

Downloading:   0%|          | 0.00/279 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/828k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

  0%|          | 0/655 [00:00<?, ?ba/s]

  0%|          | 0/73 [00:00<?, ?ba/s]

In [49]:
dataset_train.save_to_disk('gazeta/train')
dataset_test.save_to_disk('gazeta/test')

Saving the dataset (0/1 shards):   0%|          | 0/5240 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/577 [00:00<?, ? examples/s]

In [50]:
!ls

data  gazeta  sample_data  tweets.txt


In [51]:
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments


model = T5ForConditionalGeneration.from_pretrained(model_name)

Downloading:   0%|          | 0.00/766 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/977M [00:00<?, ?B/s]

In [52]:
output_dir = 'gazeta/output'

training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=20,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_accumulation_steps=1, 
    prediction_loss_only=True, 
    learning_rate=0.00001,
    evaluation_strategy='steps',
    save_steps=1000, 
    save_total_limit=1, 
    remove_unused_columns=True, 
    run_name='run_gazeta', 
    logging_steps=500, 
    eval_steps=500,
    logging_first_step=False, 
    load_best_model_at_end=True, 
    metric_for_best_model="loss", 
    greater_is_better=False 
)

In [53]:
%%time

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_test
)

trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 5240
  Num Epochs = 20
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 13100
  Number of trainable parameters = 244309248


Step,Training Loss,Validation Loss
500,8.745900,3.008487
1000,3.693100,2.714198
1500,2.302100,2.631077
2000,2.176400,2.567793
2500,2.103900,2.536541
3000,2.055100,2.516761
3500,2.026300,2.494707
4000,2.032700,2.472049
4500,1.974800,2.459195
5000,1.981100,2.443172


***** Running Evaluation *****
  Num examples = 577
  Batch size = 8
***** Running Evaluation *****
  Num examples = 577
  Batch size = 8
Saving model checkpoint to gazeta/output/checkpoint-1000
Configuration saved in gazeta/output/checkpoint-1000/config.json
Model weights saved in gazeta/output/checkpoint-1000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 577
  Batch size = 8
***** Running Evaluation *****
  Num examples = 577
  Batch size = 8
Saving model checkpoint to gazeta/output/checkpoint-2000
Configuration saved in gazeta/output/checkpoint-2000/config.json
Model weights saved in gazeta/output/checkpoint-2000/pytorch_model.bin
Deleting older checkpoint [gazeta/output/checkpoint-1000] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 577
  Batch size = 8
***** Running Evaluation *****
  Num examples = 577
  Batch size = 8
Saving model checkpoint to gazeta/output/checkpoint-3000
Configuration saved in gazeta/output/checkpoint-3000/con

CPU times: user 54min 57s, sys: 5min 53s, total: 1h 50s
Wall time: 1h 2min 43s


TrainOutput(global_step=13100, training_loss=2.2586243339713294, metrics={'train_runtime': 3763.3348, 'train_samples_per_second': 27.848, 'train_steps_per_second': 3.481, 'total_flos': 1.04350215168e+16, 'train_loss': 2.2586243339713294, 'epoch': 20.0})

In [54]:
trainer.save_model(output_dir + '/model')

Saving model checkpoint to gazeta/output/model
Configuration saved in gazeta/output/model/config.json
Model weights saved in gazeta/output/model/pytorch_model.bin


In [55]:
INX = 100
print("SUMMARY: | {}".format(dataset_test['summary'][INX]))
print("TITLE: | {}".format(dataset_test['title'][INX]))

SUMMARY: | Российские фрегаты «Адмирал Макаров» и «Адмирал Григорович» с крылатыми ракетами «Калибр-НК» проходят через турецкие проливы Босфор и Дарданеллы в направлении Средиземного моря. Ранее эти корабли уже участвовали в военной операции РФ в Сирии и теперь направляются туда на фоне обострившейся ситуации в Идлибе.
TITLE: | Снова в Сирию: российские фрегаты идут в Средиземное море


In [56]:
device = "cuda"

In [57]:
import torch

input_text = dataset_test['summary'][INX]

with torch.no_grad():
    tokenized_text = tokenizer(input_text, truncation=True, padding=True, return_tensors='pt')

    source_ids = tokenized_text['input_ids'].to(device, dtype = torch.long)
    source_mask = tokenized_text['attention_mask'].to(device, dtype = torch.long)

    generated_ids = model.generate(
        input_ids = source_ids,
        attention_mask = source_mask, 
        max_length=512,
        num_beams=7,
        temperature = 1.3,
        repetition_penalty=1, 
        length_penalty=1, 
        early_stopping=True,
        no_repeat_ngram_size=2
    )

    pred = tokenizer.decode(generated_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

print("\noutput:\n" + pred)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



output:
«Адмирал Макаров» и «Габригович» пошли в Средиземноморье


In [59]:
INX = 11
print("SUMMARY: | {}".format(dataset_test['summary'][INX]))
print("TITLE: | {}".format(dataset_test['title'][INX]))

input_text = dataset_test['summary'][INX]

with torch.no_grad():
    tokenized_text = tokenizer(input_text, truncation=True, padding=True, return_tensors='pt')

    source_ids = tokenized_text['input_ids'].to(device, dtype = torch.long)
    source_mask = tokenized_text['attention_mask'].to(device, dtype = torch.long)

    generated_ids = model.generate(
        input_ids = source_ids,
        attention_mask = source_mask, 
        max_length=512,
        num_beams=7,
        temperature = 1.3,
        repetition_penalty=1, 
        length_penalty=1, 
        early_stopping=True,
        no_repeat_ngram_size=2
    )

    pred = tokenizer.decode(generated_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

print("\noutput:\n" + pred)

SUMMARY: | Юридическая компания из США подала в суд на Apple и Samsung за выпуск смартфонов, чье излучение якобы превышает установленные законом нормы. По версии истца, вендоры подвергают здоровье своих клиентов опасности. Между тем Федеральная комиссия по связи США провела собственное расследование уровня излучения популярных гаджетов и поделилась его результатами.
TITLE: | Опасное излучение: на Apple и Samsung подали в суд

output:
Apple и Samsung подвергаются опасности
